In [3]:
import time
import pandas as pd

In [4]:
# Selenium imports
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
def init_driver(path):    #path = '../driver/chromedriver.exe'
    # Initialize crome driver
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--profile-directory=Default')
    chrome_options.add_argument('--incognito')
    chrome_options.add_argument('--disable-pligins-discovery')
    chrome_options.add_argument('--start-maximized')
    
    browser = webdriver.Chrome(executable_path=path, chrome_options=chrome_options)
    
    return browser

In [ ]:
def search_jobs(job_name:str, job_location:str, num_jobs:int, sleep_time):
    
    # initialize webdriver
    browser = init_driver(path = ) # driver =  browser  path = '../driver/chromedriver.exe'
    url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=data+science&locT=N&locId=96&jobType=&context=Jobs&sc.keyword=data+science&dropdown=0'
    browser.get(url)
    
    job_title = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Job Title, Keywords, or Company']")))
    location = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[placeholder='Location']")))
    
    job_title.clear()
    location.clear()
    time.sleep(5)  # wait for 5 seconds
    
    job_title.send_keys(job_name)
    location.send_keys(job_location)
    time.sleep(3)
    
    press_submit = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
    jobs = []
    
    while len(jobs) < num_jobs:
###################CLOSE SIGN UP POP-UP#############################################        
        time.sleep(15)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            browser.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            browser.find_element_by_css_selector('[alt="Close"]').click() #clicking to the X.
            print(' x out worked')
        except NoSuchElementException:
            print(' x out failed')
            pass
####################################################################################        
        # Going through each job in this page
        job_boxes = browser.find_elements_by_class_name("jl")
        for job_box in job_boxes:
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
                
            job_box.click()
            time.sleep(1)
            
            collected_successfully = False
            while not collected_successfully:
                try:
                    company_name = browser.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = browser.find_element_by_xpath('.//div[@class="location"]').text
                    job_nu_naam = browser.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = browser.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text  # The role para on webpage
                    collected_successfully = True
                except:
                    time.sleep(5)
                    
                try:
                    salary_estimate = browser.find_element_by_xpath('.//span[@class="css-1uyte9r css-hca4ks e1wijj242"]').text  # css-1uyte9r css-hca4ks e1wijj242
                except NoSuchElementException:
                    salary_estimate = -1
                    
                try:
                    rating = browser.find_element_by_xpath('.//span[@class="rating"]').text
                except NoSuchElementException:
                    rating = -1
                    
                if verbose:
                    print(f"Job Title: {job_nu_naam}")
                    print(f"Salary Estimate: {salart_estimate}")
                    print(f"Job Description: {job_description}")
                    print(f"Rating: {rating}")
                    print(f"Company Name: {company_name}")
                    print(f"Location: {location}")